<a href="https://colab.research.google.com/github/doricardo/covid19-rio-de-janeiro/blob/master/covid19_prefeitura_rio_de_janeiro_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
!pip install investpy
!pip install plotly_express -U
!pip install plotly==4.14.3
!pip install unidecode

     |████████████████████████████████| 245kB 9.0MB/s 
     |████████████████████████████████| 4.4MB 6.6MB/s 
     |████████████████████████████████| 5.5MB 50.7MB/s 
     |████████████████████████████████| 512kB 49.0MB/s 
  Created wheel for investpy: filename=investpy-1.0.3-cp37-none-any.whl size=4466737 sha256=24a08c07a22515f6a0e78a2ab2589023d75f56c9c93dfd964fd89a5add8a649a
  Stored in directory: /root/.cache/pip/wheels/ef/ef/31/cd22f970a59025c6d6dee56091b1f538d850e932697db1950e
Successfully built investpy
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9


     |████████████████████████████████| 13.2MB 264kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import urllib
import pandas as pd, datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from unidecode import unidecode

from IPython.display import display, HTML, display_html

root_path = os.getcwd()
save_path = os.path.join(os.getcwd(), root_path + "/_includes/")

In [ ]:
#Predição do encoding do arquivo
def predict_encoding(file_path, n_lines=20):
    '''Predict a file's encoding using chardet'''
    import chardet
    with open(file_path, 'rb') as f:
        rawdata = b''.join([f.readline() for _ in range(n_lines)])

    return chardet.detect(rawdata)['encoding']

def remove_acento(palavra):
    return(unidecode(palavra))

def ordem_faixa_etaria(name):
    #name = name.lower()
    if name == 'De 0 a 9': return 1
    if name == 'De 10 a 19': return 2
    if name == 'De 20 a 29': return 3
    if name == 'De 30 a 39': return 4
    if name == 'De 40 a 49': return 5
    if name == 'De 50 a 59': return 6
    if name == 'De 60 a 69': return 7
    if name == 'De 70 a 79': return 8
    if name == 'De 80 a 89': return 9
    if name == 'De 90 a 100': return 10
    if name == 'De 100 a 109': return 11   

In [ ]:
#Populacao da cidade do Rio de Janeiro - fonte: Data.rio
remote_file_name = "http://pcrj.maps.arcgis.com/sharing/rest/content/items/35e67befce5e4519b95cba07e4d308e9/data"
populacao_file = os.path.join("populacao_rj.xls")
urllib.request.urlretrieve(remote_file_name, populacao_file) 

('populacao_rj.xls', <http.client.HTTPMessage at 0x7ff5af907350>)

In [ ]:
df_populacao = pd.read_excel(populacao_file, header=3)

In [ ]:
df_populacao.columns = ['bairro','populacao','domicilios']
df_populacao['bairro'] = df_populacao['bairro'].str.upper()
df_populacao.drop(df_populacao[df_populacao['bairro'] == 'TOTAL'].index, inplace=True)
df_populacao.dropna(inplace=True)
df_populacao['bairro'] = df_populacao['bairro'].apply(remove_acento)
df_populacao = df_populacao.sort_values('populacao', ascending=False).set_index('bairro')

In [ ]:
#df_populacao['populacao'] / df_populacao['domicilios']
df_populacao['moradores_domicilio'] = round(df_populacao['populacao'] / df_populacao['domicilios'],2)
#df_populacao.sort_values('moradores_domicilio', ascending=False).head(40)
#--df_populacao.moradores_domicilio.mean()

In [ ]:
#Dataset covid-19 da cidade do Rio de Janeiro - fonte: Data.rio
remote_file_name = "http://pcrj.maps.arcgis.com/sharing/rest/content/items/ae85fc84a9b244108d96c7072be4d3d3/data"
covid_file = os.path.join("covid19_rj.csv")
urllib.request.urlretrieve(remote_file_name, covid_file) 

('covid19_rj.csv', <http.client.HTTPMessage at 0x7ff5ad1b0e50>)

In [ ]:
df_covid = pd.read_csv(covid_file,sep=';',encoding=predict_encoding(covid_file))

In [ ]:
print(predict_encoding(covid_file))

utf-8


In [ ]:
print(df_covid.shape, df_populacao.shape)

(213445, 13) (162, 3)


In [ ]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213445 entries, 0 to 213444
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   classificação_final    213445 non-null  object
 1   dt_notific             213402 non-null  object
 2   dt_inicio_sintomas     213333 non-null  object
 3   bairro_resid_estadia   213445 non-null  object
 4   ap_residencia_estadia  213445 non-null  object
 5   sexo                   213445 non-null  object
 6   faixa_etária           213445 non-null  object
 7   evolução               213445 non-null  object
 8   dt_obito               201502 non-null  object
 9   raça/cor               213445 non-null  object
 10  Data_atualização       213445 non-null  object
 11  sistema                213445 non-null  object
 12  id                     213445 non-null  object
dtypes: object(13)
memory usage: 21.2+ MB


In [ ]:
#redefinindo tamanho da área para trabalho
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', df_covid.shape[1]+3)

In [ ]:
# Renomeando as colunas para melhor entendimento
df_covid.columns = ['classificacao', 'dt_notificacao', 'dt_inicio_sintomas','bairro', 'area_planejamento', 
              'sexo', 'faixa_etaria', 'evolucao', 'dt_obito', 'raca_cor', 'data_atualizacao','sistema','id']

In [ ]:
print("Última atualização: {}".format(df_covid.data_atualizacao.describe()[2]))
ate = df_covid.data_atualizacao.describe()[2]#.strftime("%d/%m")

Última atualização: 03/15/21


In [ ]:
#Limpeza e padronização de dados inválidos
df_covid = df_covid.replace('Missing', np.NaN)
df_covid = df_covid.replace('Ignorado','INDEFINIDO')
df_covid = df_covid.replace('N/D','INDEFINIDO')
df_covid = df_covid.replace('#N/D','INDEFINIDO')
df_covid = df_covid.replace('ND','INDEFINIDO')
df_covid = df_covid.replace('óbito','obito')

In [ ]:
df_covid

,classificacao,dt_notificacao,dt_inicio_sintomas,bairro,area_planejamento,sexo,faixa_etaria,evolucao,dt_obito,raca_cor,data_atualizacao,sistema,id
0,CONFIRMADO,09/18/20,09/03/20,PACIENCIA,5.3,M,De 50 a 59,OBITO,09/22/20,Preta,03/15/21,SIVEP,OBITO
1,CONFIRMADO,11/25/20,11/02/20,BARRA DA TIJUCA,4.0,M,De 80 a 89,OBITO,01/12/21,Branca,03/15/21,SIVEP,OBITO
2,CONFIRMADO,05/06/20,05/06/20,CACHAMBI,3.2,M,De 70 a 79,OBITO,05/14/20,INDEFINIDO,03/15/21,SIVEP,OBITO
3,CONFIRMADO,11/12/20,11/02/20,BARRA DA TIJUCA,4.0,M,De 70 a 79,OBITO,12/14/20,Branca,03/15/21,SIVEP,OBITO
4,CONFIRMADO,06/13/20,04/26/20,MARECHAL HERMES,3.3,M,De 60 a 69,OBITO,07/24/20,INDEFINIDO,03/15/21,SIVEP,OBITO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213440,CONFIRMADO,02/18/21,02/13/21,ENGENHO NOVO,3.2,F,De 20 a 29,RECUPERADO,02/27/21,Preta,03/15/21,ESUS,ESUS
213441,CONFIRMADO,03/14/21,03/11/21,BARRA DA TIJUCA,4.0,F,De 50 a 59,ATIVO,NaN,INDEFINIDO,03/15/21,ESUS,ESUS
213442,CONFIRMADO,02/23/21,02/17/21,INHAUMA,3.2,F,De 20 a 29,RECUPERADO,03/03/21,Parda,03/15/21,ESUS,ESUS
213443,CONFIRMADO,03/14/21,03/10/21,JARDIM BOTANICO,2.1,F,De 20 a 29,ATIVO,NaN,Branca,03/15/21,ESUS,ESUS


In [ ]:
pd.to_datetime(df_covid['dt_obito'], format='%d/%m/%Y', infer_datetime_format=True)

0        2020-09-22
1        2021-01-12
2        2020-05-14
3        2020-12-14
4        2020-07-24
            ...    
213440   2021-02-27
213441          NaT
213442   2021-03-03
213443          NaT
213444          NaT
Name: dt_obito, Length: 213445, dtype: datetime64[ns]

In [ ]:
#Conversão colunas datas
df_covid['dt_notificacao'] =  pd.to_datetime(df_covid['dt_notificacao'], format='%d/%m/%Y', infer_datetime_format=True)
df_covid['dt_inicio_sintomas'] =  pd.to_datetime(df_covid['dt_inicio_sintomas'], format='%d/%m/%Y', infer_datetime_format=True)
df_covid['dt_obito'] =  pd.to_datetime(df_covid['dt_obito'], format='%d/%m/%Y', infer_datetime_format=True)
df_covid['data_atualizacao'] =  pd.to_datetime(df_covid['data_atualizacao'], format='%d/%m/%Y', infer_datetime_format=True)

In [ ]:
#dataset população e covid19
display(df_populacao.tail(), df_covid.head())

,populacao,domicilios,moradores_domicilio
bairro,,,
CAMORIM,1970.0,659.0,2.99
CIDADE UNIVERSITARIA,1556.0,534.0,2.91
CAMPO DOS AFONSOS,1365.0,375.0,3.64
JOA,818.0,251.0,3.26
GRUMARI,167.0,44.0,3.80


,classificacao,dt_notificacao,dt_inicio_sintomas,bairro,area_planejamento,sexo,faixa_etaria,evolucao,dt_obito,raca_cor,data_atualizacao,sistema,id
0,CONFIRMADO,2020-09-18,2020-09-03,PACIENCIA,5.3,M,De 50 a 59,OBITO,2020-09-22,Preta,2021-03-15,SIVEP,OBITO
1,CONFIRMADO,2020-11-25,2020-11-02,BARRA DA TIJUCA,4.0,M,De 80 a 89,OBITO,2021-01-12,Branca,2021-03-15,SIVEP,OBITO
2,CONFIRMADO,2020-05-06,2020-05-06,CACHAMBI,3.2,M,De 70 a 79,OBITO,2020-05-14,INDEFINIDO,2021-03-15,SIVEP,OBITO
3,CONFIRMADO,2020-11-12,2020-11-02,BARRA DA TIJUCA,4.0,M,De 70 a 79,OBITO,2020-12-14,Branca,2021-03-15,SIVEP,OBITO
4,CONFIRMADO,2020-06-13,2020-04-26,MARECHAL HERMES,3.3,M,De 60 a 69,OBITO,2020-07-24,INDEFINIDO,2021-03-15,SIVEP,OBITO


# Resumo geral

In [ ]:
obitos  = df_covid[df_covid['evolucao'] == 'obito'].classificacao.count()
ativos  = df_covid[df_covid['evolucao'] == 'ativo'].classificacao.count()
recuperados  = df_covid[df_covid['evolucao'] == 'recuperado'].classificacao.count()
total = obitos + ativos + recuperados

In [ ]:
fig = go.Figure(data=[go.Table(header=dict(values=['Última atualização', 'Total de casos', 'Recuperados', 'Ativos', 'Óbitos' ]),
                 cells=dict(values=[ate, total, recuperados, ativos, obitos ]))
                     ])
fig.update_layout(
    autosize=True,
    height=100,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=10,
        pad=0
    ),
)

fig.show()
fig.write_html(os.path.join(save_path, 'resumo-geral.html'))

In [ ]:
# Merge dadasets populacao 2 covid-19

In [ ]:
df = None
if df is not None:
    del df
    
df = pd.merge(df_covid, df_populacao, how='left', on='bairro')

# Casos por bairro

In [ ]:
df_bairro = df.groupby(['bairro','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['bairro'],columns=['evolucao']).reset_index()
df_bairro.columns = ['bairro','ativo','obito','recuperado']
df_bairro['recuperado'] = df_bairro.recuperado.fillna(0)
df_bairro['ativo'] = df_bairro.ativo.fillna(0)
df_bairro['obito'] = df_bairro.obito.fillna(0)
df_bairro['total'] = df_bairro.ativo + df_bairro.recuperado + df_bairro.obito
df_bairro['total_acumulado'] = df_bairro.total.cumsum()

df_bairro = pd.merge(df_bairro, df_populacao, how='left', on='bairro')
df_bairro = df_bairro.drop(['domicilios', 'moradores_domicilio'], axis=1)
df_bairro.dropna(inplace=True)

print("Taxa de letalidade em {}: {}".format(ate, (df_bairro.sum().obito / df_bairro.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_bairro.sum().recuperado / df_bairro.sum().total) * 100))

Taxa de letalidade em 03/15/21: 9.487066807018978
Taxa de recuperação em 03/15/21: 85.02948420525503


In [ ]:
df[df.dt_notificacao.isnull()]

,classificacao,dt_notificacao,dt_inicio_sintomas,bairro,area_planejamento,sexo,faixa_etaria,evolucao,dt_obito,raca_cor,data_atualizacao,sistema,id,populacao,domicilios,moradores_domicilio
47052,CONFIRMADO,NaT,2020-11-08,PENHA CIRCULAR,3.1,F,De 20 a 29,RECUPERADO,2020-11-22,Branca,2021-03-15,ESUS,ESUS,47816.0,15608.0,3.06
73317,CONFIRMADO,NaT,2020-11-09,BOTAFOGO,2.1,M,De 40 a 49,RECUPERADO,2020-11-23,INDEFINIDO,2021-03-15,ESUS,ESUS,82890.0,35254.0,2.35
77585,CONFIRMADO,NaT,2020-11-23,CAMPO DOS AFONSOS,5.1,F,De 70 a 79,RECUPERADO,2020-12-07,Parda,2021-03-15,ESUS,ESUS,1365.0,375.0,3.64
83295,CONFIRMADO,NaT,2020-07-15,GRAJAU,2.2,F,De 40 a 49,RECUPERADO,2020-07-29,Branca,2021-03-15,ESUS,ESUS,38671.0,14353.0,2.69
86929,CONFIRMADO,NaT,2020-05-17,PARQUE ANCHIETA,3.3,F,De 40 a 49,RECUPERADO,2020-05-31,Parda,2021-03-15,ESUS,ESUS,26212.0,8479.0,3.09
87610,CONFIRMADO,NaT,2020-04-27,BARRA DA TIJUCA,4.0,M,De 60 a 69,RECUPERADO,2020-05-11,Branca,2021-03-15,ESUS,ESUS,136831.0,51427.0,2.66
89886,CONFIRMADO,NaT,2020-04-20,INDEFINIDO,INDEFINIDO,F,De 30 a 39,RECUPERADO,2020-05-04,Preta,2021-03-15,ESUS,ESUS,NaN,NaN,NaN
90147,CONFIRMADO,NaT,2020-04-03,VIDIGAL,2.1,F,De 30 a 39,RECUPERADO,2020-04-17,Parda,2021-03-15,ESUS,ESUS,12797.0,4311.0,2.97
92749,CONFIRMADO,NaT,2020-04-20,MARACANA,2.2,M,De 30 a 39,RECUPERADO,2020-05-04,INDEFINIDO,2021-03-15,ESUS,ESUS,25256.0,9873.0,2.56
94851,CONFIRMADO,NaT,2020-05-07,OLARIA,3.1,F,De 60 a 69,RECUPERADO,2020-05-21,Branca,2021-03-15,ESUS,ESUS,57514.0,19827.0,2.90


In [ ]:
# Se data notificação inexistente vou atualizar com a data de inicio dos sintomas, se essa tambem for inexistente
# vou atualizar com a data de atualizacao
df['dt_notificacao'].loc[df.dt_notificacao.isnull()] = df['dt_inicio_sintomas'].loc[df.dt_notificacao.isnull()]
df['dt_notificacao'].loc[df.dt_notificacao.isnull()] = df['data_atualizacao'].loc[df.dt_notificacao.isnull()]
# Inlcuir Coluna semana
df['semana'] = df.apply(lambda row: row['dt_notificacao'] - dt.timedelta(days=row['dt_notificacao'].weekday()), axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Distribuição por bairros

In [ ]:
df_bairro  = df.groupby(['bairro','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['bairro'],columns=['evolucao']).reset_index()

df_bairro.columns = ['bairro','ativo','obito','recuperado']
df_bairro['recuperado'] = df_bairro.recuperado.fillna(0).astype(int)
df_bairro['ativo'] = df_bairro.ativo.fillna(0).astype(int)
df_bairro['obito'] = df_bairro.obito.fillna(0).astype(int)
df_bairro['total'] = df_bairro.ativo + df_bairro.recuperado + df_bairro.obito

df_bairro = pd.merge(df_bairro, df_populacao, how='left', on='bairro')
df_bairro = df_bairro.drop(['domicilios', 'moradores_domicilio'], axis=1)
df_bairro.dropna(inplace=True)

df_bairro['taxa_letalidade'] = (df_bairro.obito / df_bairro.total) * 100
df_bairro['taxa_recuperacao'] = (df_bairro.recuperado / df_bairro.total) * 100
df_bairro['obito_1000_habitantes'] = round((df_bairro.obito * 1000)/ df_bairro.populacao).astype(int)
df_bairro['casos_1000_habitantes'] = round((df_bairro.total * 1000)/ df_bairro.populacao).astype(int)

df_bairro['populacao'] = df_bairro.populacao.astype(int)

print("Taxa de letalidade em {}: {}".format(ate, (df_bairro.sum().obito / df_bairro.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_bairro.sum().recuperado / df_bairro.sum().total) * 100))
df_bairro = df_bairro.set_index('bairro')

Taxa de letalidade em 03/15/21: 9.487066807018978
Taxa de recuperação em 03/15/21: 85.02948420525503


In [ ]:
df_bairro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157 entries, ABOLICAO to ZUMBI
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ativo                  157 non-null    int64  
 1   obito                  157 non-null    int64  
 2   recuperado             157 non-null    int64  
 3   total                  157 non-null    int64  
 4   populacao              157 non-null    int64  
 5   taxa_letalidade        157 non-null    float64
 6   taxa_recuperacao       157 non-null    float64
 7   obito_1000_habitantes  157 non-null    int64  
 8   casos_1000_habitantes  157 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 12.3+ KB


In [ ]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [ ]:
display_side_by_side([df_bairro[['total','populacao']].sort_values('total', ascending=False)[:30],
                     df_bairro[['total','populacao','casos_1000_habitantes']].sort_values('casos_1000_habitantes', ascending=False)[:30]],
                    ['Casos  totais - Top20 bairros', 'Casos por 1000 habitantes - Top20 bairros'])

#fig.show()
#fig.write_html(os.path.join(save_path, 'casos-por-habitantes-tabela.html'))

,total,populacao
bairro,,
BARRA DA TIJUCA,9647,136831
TIJUCA,8477,163805
COPACABANA,7808,146392
CAMPO GRANDE,7512,328370
SANTA CRUZ,5443,217333
BOTAFOGO,5056,82890
RECREIO DOS BANDEIRANTES,5010,84224
JACAREPAGUA,4846,157326
BANGU,4682,220552


In [ ]:
display_side_by_side([df_bairro[['obito','populacao']].sort_values('obito', ascending=False)[:50],df_bairro[['obito','populacao','obito_1000_habitantes']].sort_values('obito_1000_habitantes', ascending=False)[:50]],
                    ['Óbitos totais - Top20 bairros', 'Óbitos por 1000 habitantes - Top20 bairros'])

,obito,populacao
bairro,,
CAMPO GRANDE,1135,328370
BANGU,809,220552
TIJUCA,668,163805
COPACABANA,642,146392
REALENGO,602,180123
SANTA CRUZ,591,217333
BARRA DA TIJUCA,431,136831
TAQUARA,409,102126
IRAJA,320,96382


## Casos por mil habitantes - Top10 Bairros

In [ ]:
df_bairro_top10 = df_bairro.sort_values('total', ascending=False)[:10]
fig = px.scatter(df_bairro_top10, x="populacao", y="total", size="obito", color=df_bairro_top10.index,
                hover_name=df_bairro_top10.index, log_x=False, size_max=60,
                template="simple_white",
                title="Casos por habitantes - Top10 Bairros")

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'casos-por-habitantes.html'))

In [ ]:
df_bairro_diario  = df.groupby(['bairro','semana','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['bairro','semana'], columns=['evolucao']).reset_index()

df_bairro_diario.columns = ['bairro','data','ativo','obito','recuperado']
df_bairro_diario['recuperado'] = df_bairro_diario.recuperado.fillna(0)
df_bairro_diario['ativo'] = df_bairro_diario.ativo.fillna(0)
df_bairro_diario['obito'] = df_bairro_diario.obito.fillna(0)
df_bairro_diario['total'] = df_bairro_diario.ativo + df_bairro_diario.recuperado + df_bairro_diario.obito

df_bairro_diario = pd.merge(df_bairro_diario, df_populacao, how='left', on='bairro')
df_bairro_diario = df_bairro_diario.drop(['domicilios', 'moradores_domicilio'], axis=1)
df_bairro_diario.dropna(inplace=True)

df_bairro_diario['taxa_letalidade'] = (df_bairro_diario.obito / df_bairro_diario.total) * 100
df_bairro_diario['taxa_recuperacao'] = (df_bairro_diario.recuperado / df_bairro_diario.total) * 100
df_bairro_diario['obito_1000_habitantes'] = ((df_bairro_diario.obito * 1000)/ df_bairro_diario.populacao)
df_bairro_diario['casos_1000_habitantes'] = ((df_bairro_diario.total * 1000)/ df_bairro_diario.populacao)

print("Taxa de letalidade em {}: {}".format(ate, (df_bairro_diario.sum().obito / df_bairro_diario.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_bairro_diario.sum().recuperado / df_bairro_diario.sum().total) * 100))
df_bairro_diario = df_bairro_diario.set_index('data')

df_bairro_diario[['obito','populacao','obito_1000_habitantes']].sort_values('obito_1000_habitantes', ascending=False)[:10]

df_bairro_diario.head()

Taxa de letalidade em 03/15/21: 9.487066807018978
Taxa de recuperação em 03/15/21: 85.02948420525503


,bairro,ativo,obito,recuperado,total,populacao,taxa_letalidade,taxa_recuperacao,obito_1000_habitantes,casos_1000_habitantes
data,,,,,,,,,,
2020-03-23,ABOLICAO,0.0,0.0,1.0,1.0,11356.0,0.000000,100.000000,0.000000,0.088059
2020-03-30,ABOLICAO,0.0,1.0,2.0,3.0,11356.0,33.333333,66.666667,0.088059,0.264178
2020-04-06,ABOLICAO,0.0,2.0,9.0,11.0,11356.0,18.181818,81.818182,0.176118,0.968651
2020-04-13,ABOLICAO,0.0,4.0,3.0,7.0,11356.0,57.142857,42.857143,0.352237,0.616414
2020-04-20,ABOLICAO,0.0,5.0,5.0,10.0,11356.0,50.000000,50.000000,0.440296,0.880592


In [ ]:
#bairros = df_bairro_top10.index
bairros = df_bairro.sort_values('total', ascending=False)[:10].index
df_bairro_1 = df_bairro_diario[df_bairro_diario['bairro'].isin(bairros)]

desde = df_bairro_1.index.max().strftime("%d/%m")
fig = px.line(df_bairro_1, 
              width=1280,
              x=df_bairro_1.index, 
              y=df_bairro_1["obito"],
              line_shape="spline",
              color=df_bairro_1["bairro"],
              template="simple_white",
              title='Evolução dos óbitos nos bairros com mais casos de COVID-19')

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'evolucao-casos-obitos-bairros-top10.html'))

## Letalidade x Recuperação geral desde o início da pandemia na cidade

In [ ]:
df_dia  = df.groupby(['dt_notificacao','evolucao']).classificacao.count().reset_index().\
pivot_table(index=['dt_notificacao'],columns=['evolucao']).reset_index()

df_dia.columns = ['data','ativo','obito','recuperado']
df_dia['recuperado'] = df_dia.recuperado.fillna(0)
df_dia['ativo'] = df_dia.ativo.fillna(0)
df_dia['obito'] = df_dia.obito.fillna(0)
df_dia['total'] = df_dia.ativo + df_dia.recuperado + df_dia.obito

df_dia['obito_acumulado'] = df_dia.obito.cumsum()
df_dia['recuperado_acumulado'] = df_dia.recuperado.cumsum()
df_dia['total_acumulado'] = df_dia.total.cumsum()
df_dia['aumento'] = df_dia.obito_acumulado - df_dia.obito_acumulado.shift(+1)
df_dia['aceleracao'] = df_dia.aumento - df_dia.aumento.shift(+1)

df_dia['taxa_letalidade'] = (df_dia.obito_acumulado / df_dia.total_acumulado) * 100
df_dia['taxa_recuperacao'] =(df_dia.recuperado_acumulado / df_dia.total_acumulado) * 100

print("Taxa de letalidade em {}: {}".format(ate, (df_dia.sum().obito / df_dia.sum().total) * 100))
print("Taxa de recuperação em {}: {}".format(ate, (df_dia.sum().recuperado / df_dia.sum().total) * 100))
df_dia = df_dia.set_index('data')

Taxa de letalidade em 03/15/21: 9.058071165874113
Taxa de recuperação em 03/15/21: 85.61362411862541


In [ ]:
df_dia.tail(30)

,ativo,obito,recuperado,total,obito_acumulado,recuperado_acumulado,total_acumulado,aumento,aceleracao,taxa_letalidade,taxa_recuperacao
data,,,,,,,,,,,
2021-02-14,10.0,13.0,39.0,62.0,18937.0,176820.0,204634.0,13.0,2.0,9.254083,86.407928
2021-02-15,27.0,31.0,301.0,359.0,18968.0,177121.0,204993.0,31.0,18.0,9.252999,86.403438
2021-02-16,14.0,13.0,133.0,160.0,18981.0,177254.0,205153.0,13.0,-18.0,9.252119,86.400881
2021-02-17,49.0,28.0,446.0,523.0,19009.0,177700.0,205676.0,28.0,15.0,9.242206,86.398024
2021-02-18,25.0,52.0,360.0,437.0,19061.0,178060.0,206113.0,52.0,24.0,9.247840,86.389505
2021-02-19,45.0,23.0,343.0,411.0,19084.0,178403.0,206524.0,23.0,-29.0,9.240573,86.383665
2021-02-20,10.0,12.0,87.0,109.0,19096.0,178490.0,206633.0,12.0,-11.0,9.241505,86.380201
2021-02-21,14.0,10.0,47.0,71.0,19106.0,178537.0,206704.0,10.0,-2.0,9.243169,86.373268
2021-02-22,51.0,28.0,474.0,553.0,19134.0,179011.0,207257.0,28.0,18.0,9.232016,86.371510


## Total de casos e óbitos capital Rio de Janeiro

In [ ]:
desde = df_dia.index.min().strftime("%d/%m")
fig = px.line(df_dia, 
                            width=1280,
              x=df_dia.index, 
              y=["aumento"],
              line_shape="spline",
              template="simple_white",
              title='Evolução de casos e óbitos' )

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'evolucao-casos-obitos.html'))

In [ ]:
desde = df_dia.index.min().strftime("%d/%m")
fig = px.line(df_dia, 
              x=df_dia.index, 
              y=["taxa_letalidade"],
              line_shape="spline",
              template="xgridoff",
              title='Taxa de letalidade' )

fig.update_yaxes(showticklabels=True, visible=False)
fig.update_xaxes(showticklabels=True, visible=True, title='')

fig.show()
fig.write_html(os.path.join(save_path, 'taxa-letalidade.html'))